In [1]:
import tkinter as tk

def rabin_karp(pattern, text, q, output_text):
    M = len(pattern)
    N = len(text)
    p = 0  # hash value for pattern
    t = 0  # hash value for text
    d = 256  # number of characters in the input alphabet
    h = pow(d, M-1) % q  # hash for sliding

    # Calculate the hash value of pattern and the first window of text
    for i in range(M):
        p = (d * p + ord(pattern[i])) % q
        t = (d * t + ord(text[i])) % q

    iterations = 0
    for i in range(N - M + 1):
        iterations += 1
        # Check if the hash values match, if they do, check character by character
        if p == t:
            superior_hit = True
            for j in range(M):
                if text[i + j] != pattern[j]:
                    output_text.insert(tk.END, f"i: {i}, j: {j}, Hit: NO\n")
                    superior_hit = False
                    break
            if superior_hit:
                output_text.insert(tk.END, f"Pattern found at index: {i}, i: {i}, j: {j}, Hit: YES\n")
        
        # Calculate hash value for next window of text
        if i < N - M:
            t = (d * (t - ord(text[i]) * h) + ord(text[i + M])) % q
            # We might get negative value of t, converting it to positive
            if t < 0:
                t = t + q

    output_text.insert(tk.END, f"\nTotal characters: {iterations}\n")

def naive_string_matching(pattern, text, output_text):
    m = len(pattern)
    n = len(text)
    occurrences = []
    iterations = 0

    for i in range(n - 2):
        match = True
        for j in range(m):
            iterations += 1
            if text[i + j] != pattern[j]:
                output_text.insert(tk.END, f"i: {i}, j: {j}, Hit: NO\n")
                match = False
                break
        if match:
            output_text.insert(tk.END, f"Pattern found at index: {i}, i: {i}, j: {j}, Hit: YES\n")
            occurrences.append(i)

    output_text.insert(tk.END, f"\nTotal characters: {n}\n")  # Update to reflect iterations equal to text length

    if occurrences:
        output_text.insert(tk.END, f"\nPattern found at indices: {', '.join(map(str, occurrences))}\n")
    else:
        output_text.insert(tk.END, "Pattern not found in the text.\n")


def kmp_string_matching(pattern, text, output_text):
    m = len(pattern)
    n = len(text)
    occurrences = []
    iterations = 0

    lps = compute_lps_array(pattern)
    i = 0  
    j = 0  

    while i < n:
        iterations += 1
        if pattern[j] == text[i]:
            i += 1
            j += 1

            if j == m:
                output_text.insert(tk.END, f"Pattern found at index: {i - j}, i: {i}, j: {j}, Hit: YES\n")
                occurrences.append(i - j)
                j = lps[j - 1]
        else:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1

    output_text.insert(tk.END, f"\nTotal Characters: {n}\n")  # Update to reflect iterations equal to text length

    if occurrences:
        output_text.insert(tk.END, f"\nPattern found at indices: {', '.join(map(str, occurrences))}\n")
    else:
        output_text.insert(tk.END, "Pattern not found in the text.\n")

def compute_lps_array(pattern):
    m = len(pattern)
    lps = [0] * m
    length = 0
    i = 1

    while i < m:
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps

def on_submit():
    pattern = pattern_entry.get()
    text = text_entry.get()
    q = q_entry.get()

    if q.lower() == 'q':
        q = 101  # Prime number for hash calculation

    output_text.delete(1.0, tk.END)

    if algorithm_var_rabin.get():
        rabin_karp(pattern, text, int(q), output_text)
    if algorithm_var_naive.get():
        naive_string_matching(pattern, text, output_text)
    if algorithm_var_kmp.get():
        kmp_string_matching(pattern, text, output_text)

# Create GUI
root = tk.Tk()
root.title("String Matching Algorithms")
root.geometry("450x450")

# Pattern entry
pattern_label = tk.Label(root, text="Pattern:")
pattern_label.grid(row=0, column=0, sticky="w")
pattern_entry = tk.Entry(root)
pattern_entry.grid(row=0, column=1)

# Text entry
text_label = tk.Label(root, text="Text:")
text_label.grid(row=1, column=0, sticky="w")
text_entry = tk.Entry(root)
text_entry.grid(row=1, column=1)

# Hash value entry for Rabin-Karp
q_label = tk.Label(root, text="Hash value 'q' (for Rabin-Karp):")
q_label.grid(row=2, column=0, sticky="w")
q_entry = tk.Entry(root)
q_entry.grid(row=2, column=1)

# Choose Algorithm label
algorithm_label = tk.Label(root, text="Choose Algorithm:")
algorithm_label.grid(row=3, column=0, sticky="w")

# Checkbox options for algorithms
algorithm_var_rabin = tk.BooleanVar(root)
algorithm_var_rabin.set(True)
checkbox_rabin = tk.Checkbutton(root, text="Rabin-Karp", variable=algorithm_var_rabin)
checkbox_rabin.grid(row=4, column=0, sticky="w")

algorithm_var_naive = tk.BooleanVar(root)
algorithm_var_naive.set(False)
checkbox_naive = tk.Checkbutton(root, text="Naive", variable=algorithm_var_naive)
checkbox_naive.grid(row=4, column=1, sticky="w")

algorithm_var_kmp = tk.BooleanVar(root)
algorithm_var_kmp.set(False)
checkbox_kmp = tk.Checkbutton(root, text="Knuth-Morris-Pratt", variable=algorithm_var_kmp)
checkbox_kmp.grid(row=4, column=2, sticky="w")

# Submit button
submit_button = tk.Button(root, text="Submit", command=on_submit)
submit_button.grid(row=5, column=0, columnspan=3)

# Output text
output_text = tk.Text(root, height=10, width=50)
output_text.grid(row=6, column=0, columnspan=3)

root.mainloop()